# Phase 1A: Network Foundation and Migration (Intentional Double NAT)

**Date:** 2026-01-19  
**Estimated Time:** 2-3 hours  
**Prerequisites:** `phase1-preflight-checklist.md` complete; maintenance window scheduled

## Scope

This phase configures the physical topology, router settings, SSIDs and bands, DHCP scopes and reservations, device migration, camera internet blocking, and validation of the intentional double NAT design.

This phase does NOT include endpoint OS hardening or Home Assistant security changes (Phase 1B).

## Architecture Contract (Non-Negotiable)

- Two identical Verizon CR1000A routers.
- Router #2 stays in router mode (never AP mode).
- Router #2 WAN connects to Router #1 LAN (LAN -> WAN).
- Two subnets only:
  - Trusted: 192.168.1.0/24 (Router #1)
  - IoT/Cameras: 192.168.2.0/24 (Router #2)
- **NAT does NOT provide isolation by itself; it only reduces unsolicited inbound traffic.**
- Endpoint enforcement (host firewalls) is the primary isolation control.
- This architecture enables control and visibility, not magic isolation.
- Trusted can initiate outbound to IoT/Cameras; IoT can respond.
- Home Assistant + Scrypted live on the Mac mini (must be dual-homed; see below).
- HomeKit hubs stay on Router #1 (mDNS dependency).
- No VLANs, no AP mode, no mDNS reflection in Phase 1.

## CRITICAL: Mac mini Dual-Homing Requirement

**The Mac mini (hosting Home Assistant + Scrypted) MUST be dual-homed:**

- **Interface 1 (Trusted):** Connected to Router #1 (192.168.1.x)
  - Provides HomeKit hub access via mDNS
  - Provides HA UI access for trusted devices
- **Interface 2 (IoT):** Connected to Router #2 (192.168.2.x)
  - Enables HA/Scrypted to directly reach IoT devices and cameras on 192.168.2.x

**Without dual-homing, Home Assistant and Scrypted CANNOT reach devices on 192.168.2.x.**

Implementation options:
1. **USB-to-Ethernet adapter:** Plug into Mac mini, connect to Router #2 LAN port.
2. **Thunderbolt/USB-C dock with Ethernet:** Use the dock's Ethernet for the second interface.
3. **Wi-Fi as second interface:** Connect Mac mini Wi-Fi to ByteMe SSID (less reliable than wired).

## Target Topology (Locked)

```
ISP
  -> Router #1 (Trusted / Control Plane) 192.168.1.1/24
     - LAN ports: Apple TV 4K, Mac mini Interface 1 (Trusted), Router #2 WAN
  -> Router #2 (IoT / Edge) WAN on 192.168.1.x
     - LAN: 192.168.2.1/24 for IoT and cameras
     - LAN ports: Mac mini Interface 2 (IoT), Hue Bridge, Aqara Hub
```

**Note:** Mac mini is dual-homed—one interface on each router.

## Target IP Plan (Locked)

### Router #1 (Trusted) - 192.168.1.0/24

- Gateway: 192.168.1.1
- DHCP dynamic pool: 192.168.1.100-192.168.1.254
- Reserved ranges:
  - 192.168.1.2-19: network infrastructure
  - 192.168.1.20-49: servers and core services
  - 192.168.1.50-99: trusted static devices

In [ ]:
# Router #1 DHCP Reservations Reference
import pandas as pd

router1_reservations = pd.DataFrame([
    {"Device": "Router #1", "IP": "192.168.1.1", "Connection": "-", "Notes": "Gateway"},
    {"Device": "Router #2 WAN", "IP": "192.168.1.10", "Connection": "DHCP reservation", "Notes": "Use 192.168.1.2-19"},
    {"Device": "Mac mini M4 (Trusted)", "IP": "192.168.1.20", "Connection": "Ethernet", "Notes": "Interface 1 for HA + Scrypted"},
    {"Device": "Home Assistant VM", "IP": "192.168.1.21", "Connection": "Virtual bridge", "Notes": "HA UI at http://192.168.1.21:8123"},
    {"Device": "Apple TV 4K (Control Room)", "IP": "192.168.1.50", "Connection": "Ethernet", "Notes": "HomeKit hub + Thread border"},
    {"Device": "Apple TV (Bedroom)", "IP": "192.168.1.51", "Connection": "Wi-Fi", "Notes": "HomeKit hub"},
    {"Device": "iPad 5th Gen", "IP": "192.168.1.52", "Connection": "Wi-Fi", "Notes": "HomeKit hub / wall panel"},
])

print("Router #1 (Trusted) DHCP Reservations:")
print(router1_reservations.to_string(index=False))

### Router #2 (IoT/Cameras) - 192.168.2.0/24

- Gateway: 192.168.2.1
- DHCP dynamic pool: 192.168.2.200-192.168.2.254
- Reserved ranges:
  - 192.168.2.2-49: infrastructure
  - 192.168.2.50-149: IoT devices
  - 192.168.2.150-199: cameras

In [ ]:
# Router #2 DHCP Reservations Reference
router2_reservations = pd.DataFrame([
    {"Device": "Router #2", "IP": "192.168.2.1", "Connection": "-", "Notes": "Gateway"},
    {"Device": "Mac mini M4 (IoT)", "IP": "192.168.2.10", "Connection": "Ethernet (USB adapter)", "Notes": "Interface 2 for HA + Scrypted"},
    {"Device": "Philips Hue Bridge", "IP": "192.168.2.50", "Connection": "Ethernet or Wi-Fi", "Notes": "IoT hub"},
    {"Device": "Aqara Hub M2", "IP": "192.168.2.51", "Connection": "Ethernet", "Notes": "IoT hub"},
    {"Device": "Echo Show 8 (Bedroom)", "IP": "192.168.2.52", "Connection": "Wi-Fi", "Notes": "IoT"},
    {"Device": "Echo Dot (Office)", "IP": "192.168.2.53", "Connection": "Wi-Fi", "Notes": "IoT"},
    {"Device": "Nest Mini (Bathroom)", "IP": "192.168.2.54", "Connection": "Wi-Fi", "Notes": "IoT"},
    {"Device": "HP Printer", "IP": "192.168.2.55", "Connection": "Wi-Fi", "Notes": "IoT"},
    {"Device": "Toshiba Microwave", "IP": "192.168.2.56", "Connection": "Wi-Fi", "Notes": "IoT"},
    {"Device": "Kitchen Camera (Tapo C110)", "IP": "192.168.2.150", "Connection": "Wi-Fi (Cameras SSID)", "Notes": "Internet blocked"},
    {"Device": "Hallway Camera (Tapo C110)", "IP": "192.168.2.151", "Connection": "Wi-Fi (Cameras SSID)", "Notes": "Internet blocked"},
    {"Device": "Entrance Camera (Tapo C110)", "IP": "192.168.2.152", "Connection": "Wi-Fi (Cameras SSID)", "Notes": "Internet blocked"},
    {"Device": "Office Camera (Tapo C110)", "IP": "192.168.2.153", "Connection": "Wi-Fi (Cameras SSID)", "Notes": "Internet blocked"},
])

print("Router #2 (IoT/Cameras) DHCP Reservations:")
print(router2_reservations.to_string(index=False))

## Step 0: Pre-Change Snapshots (Required)

Capture screenshots for rollback. Save to `01-Network/screenshots/preflight/`.

**Router #1 pages:**
- Advanced -> Network Settings -> IPv4 Address Distribution (summary and DHCP Settings)
- Advanced -> Network Settings -> DHCP Connections
- Basic -> Wi-Fi -> Primary Network (EnterAtYourOwnRisk)
- Basic -> Wi-Fi -> Guest/IoT networks (if present)
- Basic -> Wi-Fi -> Radio Management (Band Steering/SON)
- Basic -> Wi-Fi -> WPS
- Advanced -> Security & Firewall -> Port Forwarding
- Advanced -> Security & Firewall -> DMZ Host
- Advanced -> Security & Firewall -> UPnP
- Advanced -> Network Settings -> IPv6

**Router #2 pages:**
- Advanced -> Network Settings -> WAN Connection
- Advanced -> Network Settings -> LAN Settings
- Advanced -> Network Settings -> IPv4 Address Distribution (summary and DHCP Settings)
- Advanced -> Network Settings -> DHCP Connections
- Basic -> Wi-Fi -> SSIDs (ByteMe, Cameras)
- Basic -> Wi-Fi -> Radio Management (Band Steering/SON)
- Basic -> Wi-Fi -> WPS
- Advanced -> Security & Firewall -> UPnP
- Advanced -> Security & Firewall -> Port Forwarding
- Advanced -> Security & Firewall -> DMZ Host

In [ ]:
# Verification checklist for screenshots
import os

screenshot_dir = r"c:\Coding\SmartHomeHQ\01-Network\screenshots\preflight"

if os.path.exists(screenshot_dir):
    files = os.listdir(screenshot_dir)
    print(f"Screenshots found: {len(files)} files")
    for f in files:
        print(f"  - {f}")
else:
    print(f"Directory not found: {screenshot_dir}")
    print("Please create the directory and capture screenshots before proceeding.")

## Step 1: Confirm Physical Wiring (LAN -> WAN)

Goal: Router #2 WAN is connected to Router #1 LAN.

- Router #1 WAN -> ISP
- Router #1 LAN -> Router #2 WAN
- Router #2 LAN -> IoT hubs (Aqara, Hue) and any IoT Ethernet devices

**Validation:**
- Router #2 WAN receives a 192.168.1.x address from Router #1.
- Router #2 stays in router mode (not AP mode).

## Step 2: Configure Router #1 (Trusted)

Access: `http://192.168.1.1`

### 2.1 LAN and DHCP

- LAN IP: `192.168.1.1`
- Subnet mask: `255.255.255.0`
- DHCP enabled
- DHCP pool: `192.168.1.100-192.168.1.254`

### 2.2 Wi-Fi (Trusted SSID)

SSID: `EnterAtYourOwnRisk`

- Bands: 5 GHz and 6 GHz enabled
- 2.4 GHz: disabled unless a trusted device requires it
- Band Steering/SON: disabled
- Security: WPA2/WPA3 or WPA2

### 2.3 Router Hardening (Trusted)

- WPS: disabled
- UPnP: disabled
- Port forwarding: empty
- DMZ: disabled
- Remote admin: disabled
- IPv6: set intentionally (leave enabled only if you use it)

## Step 3: Configure Router #2 (IoT/Cameras)

Access: `http://192.168.2.1` (must be on Router #2 LAN or ByteMe SSID)

### 3.1 LAN and DHCP

- Router mode: required
- LAN IP: `192.168.2.1`
- Subnet mask: `255.255.255.0`
- DHCP enabled
- DHCP pool: `192.168.2.200-192.168.2.254`

### 3.2 Wi-Fi (IoT + Cameras)

SSID: `ByteMe` (IoT)
- 2.4 GHz only
- Band Steering/SON disabled
- Security: WPA2

SSID: `Cameras`
- 2.4 GHz only
- Band Steering/SON disabled
- Security: WPA2

### 3.3 Router Hardening (IoT)

- WPS: disabled
- UPnP: disabled
- Port forwarding: empty
- DMZ: disabled
- Remote admin: disabled
- IPv6: set intentionally (leave enabled only if you use it)

## Step 4: Create DHCP Reservations on Router #1 (Trusted)

Access: `http://192.168.1.1` -> DHCP Connections

- Reserve each device to match the Router #1 table above.
- Reserve Router #2 WAN to 192.168.1.10 (or another unused IP in 192.168.1.2-19).

**Validation:**
- Reserved devices show correct IPs.
- Router #2 WAN shows the reserved 192.168.1.10.

## Step 5: Create DHCP Reservations on Router #2 (IoT/Cameras)

Access: `http://192.168.2.1` -> DHCP Connections

- Reserve each IoT and camera device to match the Router #2 table above.

**Validation:**
- Reserved devices show correct IPs.
- Cameras show 192.168.2.150-153.

## Step 6: Migrate Devices to Correct Subnets

### 6.1 Trusted devices -> Router #1 (Trusted)

Keep these on Router #1:
- Mac mini (HA + Scrypted host)
- Home Assistant VM
- HomeKit hubs (Apple TVs, iPad)
- Phones, PCs, tablets

**Validation:**
- All trusted devices are on 192.168.1.x.
- Home Assistant loads at `http://192.168.1.21:8123`.

### 6.2 IoT devices -> Router #2 (ByteMe)

For each IoT device:
1. Move device Wi-Fi to `ByteMe` (or wire it to Router #2 LAN).
2. Verify it appears in Router #2 DHCP Connections.
3. Create reservation from the Router #2 table.
4. Reboot device or renew DHCP.
5. Confirm it returns to the reserved IP.
6. Verify it works in its native app and in Home Assistant.

### 6.3 Cameras -> Router #2 (Cameras SSID)

For each camera:
1. Move camera Wi-Fi to `Cameras` SSID.
2. Create reservation from the Router #2 table (192.168.2.150-153).
3. Update camera IPs in Scrypted if required.
4. Verify live stream in Scrypted.

## Step 7: Block Camera Internet Access (Router #2)

Goal: block WAN access for cameras while keeping LAN access for Scrypted.

1. Router #2 -> Parental Controls or Access Control
2. Create profile: `Cameras`
3. Set Internet/WAN access: blocked (always on)
4. Assign all camera devices (by MAC/IP) to the `Cameras` profile

**Validation:**
- Cameras still stream in Scrypted.
- Camera cloud access fails (expected).

## Step 8: Validation Tests (Required)

In [ ]:
# 8.1 Double NAT and Gateway Verification
# From a trusted device (Router #1 LAN)
!netstat -nr | grep default
# Expected: default gateway is 192.168.1.1

print("\nFrom an IoT device on Router #2, run: netstat -nr | grep default")
print("Expected: default gateway is 192.168.2.1")

In [ ]:
# 8.3 Mac mini Dual-Homing Validation (REQUIRED)
# From the Mac mini (which must be dual-homed)

print("Run these commands on the Mac mini:")
print("\n# Verify both interfaces are active")
print("ifconfig | grep 'inet '")
print("# Should show both 192.168.1.20 and 192.168.2.10")

print("\n# Test connectivity to both subnets")
print("ping -c 4 192.168.1.1    # Router #1 via Trusted interface")
print("ping -c 4 192.168.2.1    # Router #2 via IoT interface")
print("ping -c 4 192.168.2.50   # Hue Bridge via IoT interface")
print("ping -c 4 192.168.2.150  # Kitchen Camera via IoT interface")

print("\nExpected: all succeed")
print("\n** If pings to 192.168.2.x fail: The Mac mini IoT interface is not properly connected or configured. **")

In [ ]:
# 8.4 HA + Scrypted Checks
import urllib.request
import urllib.error

def check_url(url, name):
    try:
        response = urllib.request.urlopen(url, timeout=5)
        print(f"✓ {name}: Reachable (Status: {response.status})")
        return True
    except urllib.error.URLError as e:
        print(f"✗ {name}: Not reachable - {e.reason}")
        return False
    except Exception as e:
        print(f"✗ {name}: Error - {str(e)}")
        return False

print("Checking Home Assistant and Scrypted accessibility...\n")
check_url("http://192.168.1.21:8123", "Home Assistant")
# Note: Scrypted uses HTTPS with self-signed cert, so this may fail with SSL error
print("\nManually verify Scrypted at: https://192.168.1.20:10443")

## Phase 1A Completion Checklist

- [ ] Pre-change screenshots saved to `01-Network/screenshots/preflight/`
- [ ] Router #2 wired LAN -> WAN (Router #1 LAN -> Router #2 WAN)
- [ ] Router #1 LAN and DHCP set to 192.168.1.0/24 with pool 192.168.1.100-254
- [ ] Router #2 LAN and DHCP set to 192.168.2.0/24 with pool 192.168.2.200-254
- [ ] **Mac mini dual-homed: Interface on Router #1 (192.168.1.20) AND Router #2 (192.168.2.10)**
- [ ] Router #1 reservations created (Mac mini Trusted interface, HA, hubs, Router #2 WAN)
- [ ] Router #2 reservations created (Mac mini IoT interface, IoT devices, cameras)
- [ ] SSIDs set: EnterAtYourOwnRisk (trusted), ByteMe (IoT 2.4 GHz), Cameras (2.4 GHz)
- [ ] WPS, UPnP, remote admin disabled on both routers
- [ ] Cameras moved to Cameras SSID and streaming in Scrypted
- [ ] Cameras internet blocked via Access Control
- [ ] IoT devices moved to ByteMe or Router #2 LAN
- [ ] Mac mini can reach both 192.168.1.x and 192.168.2.x devices
- [ ] HA and Scrypted reachable from Trusted network
- [ ] HA/Scrypted can communicate with IoT devices on 192.168.2.x

---

**Next:** Proceed to Phase 1B notebook for endpoint hardening